In [1]:
import numpy as np
import cv2 # NumPy와 OpenCV 라이브러리 임포트
from google.colab.patches import cv2_imshow # Colab 환경에서 이미지 표시를 위한 함수 임포트
import time # 출력 속도를 조절하기 위한 time 모듈 임포트

# --- 1. 초기 설정 ---
# NOTE: Colab에서 이 코드를 실행하기 전에, 'newyork.mp4' 파일을 Colab 환경(왼쪽 파일 탐색기)에
# 반드시 업로드해야 합니다.
video_path = '/content/drive/MyDrive/두산로보틱스_딥러닝_컴퓨터비전/7기_컴퓨터비전 응용/교육생제공용/data/newyork.mp4' # 처리할 비디오 파일 경로
cap = cv2.VideoCapture(video_path) # 비디오 캡처 객체 생성

# 비디오 파일이 제대로 열렸는지 확인하는 것이 좋습니다.
if not cap.isOpened():
    print(f"오류: 비디오 파일 '{video_path}'을 열 수 없습니다. 파일을 업로드했는지 확인하세요.")
    exit() # 열 수 없으면 프로그램 종료

# 원본 코드의 'delay' 변수는 Colab에서 사용되지 않으므로 삭제합니다.

In [2]:
# 배경제거 객체 생성
# MOG2(Mixture of Gaussians, 개선 버전) 알고리즘을 사용한 배경 제거 객체 생성으로 수정.
# cv2.bgsegm의 MOG는 Colab 환경에서 종종 오류를 일으키므로, 표준 cv2.createBackgroundSubtractorMOG2를 사용합니다.
fgbg = cv2.createBackgroundSubtractorMOG2()

MOG2란?
- MOG(mixture of gaussians2)
- 각 pixel 색상변화를 가우시안 분포(정규분포) 모델링
- 배경(background: bg) 고정되어 있거나 오래 머무는 색상 >> 배경 학습
- 전경(foreground: fg): 갑자기 툭 나타난 색상 : 움직이는 물체로 판단
- 가우시안 혼합모델을 활용

In [3]:
# --- 3. 비디오 처리 루프 ---
# Colab은 GUI 환경이 아니므로, 실시간 루프(cv2.waitKey) 대신
# 특정 프레임 수만 처리하고 결과를 정적으로 표시합니다.
frame_count = 0
MAX_FRAMES_TO_PROCESS = 100 # 테스트를 위해 최대 처리 프레임 수를 100으로 제한
DISPLAY_EVERY_N_FRAMES = 20 # 20 프레임마다 결과 출력

print(f"비디오 처리 시작 (최대 {MAX_FRAMES_TO_PROCESS} 프레임)...")

# 여기서 잠깐?
# 30 fps(frame per second) 대략 3초 분량

while cap.isOpened() and frame_count < MAX_FRAMES_TO_PROCESS:
    ret, frame = cap.read()
    # 동영상에서 한 프레임(사진 1장)읽기 (ret:성공여부, frame: 실제 이미지)

    if not ret:
        break

    # 배경제거 마스크 계산
    # frame 에서 배경(검정 0) 제외 >> 움직이는 객체 (전경)만 흰색(255)으로 표시한 마스크(흑백마스크) 생성
    fgmask = fgbg.apply(frame)
    # 마스크 값
    # 0(검정)>> 배경, 움직이는 않는 부분
    # 255(흰색)>> 전경, 움직이는 개체
    # 127(회색)>> MOG2 가 가진 특별 기능 (그림자)

    # 특정 간격의 프레임만 표시 >> 작동 확인
    if frame_count % DISPLAY_EVERY_N_FRAMES == 0:
      # 원본 프레임 표시
      cv2_imshow(frame)
      # 전경 마스크 표시
      cv2_imshow(fgmask)

      time.sleep(1)

    frame_count += 1

cap.release()
# cv2.destroyAllWindows()

Output hidden; open in https://colab.research.google.com to view.

In [4]:
# eos